In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [2]:
# Installing Required Libraries
%pip install python-docx
%pip install python-pptx
%pip install PyPDF2
%pip install langchain
%pip install langchain_community
%pip install langchain_google_genai
%pip install langchain_text_splitters
%pip install sentence-transformers
%pip install faiss-cpu
%pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langcha

In [3]:
# necessary Imports
from docx import Document
from PyPDF2 import PdfReader
from pptx import Presentation
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [4]:
%%writefile app.py
import streamlit as st
import os
import cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from PyPDF2 import PdfReader
from pptx import Presentation
from docx import Document

# Streamlit App Interface
st.title("Document-Based Q&A System")

# Define layout columns
col1, col2, col3 = st.columns(3)

# Left Column (Upload PDF)
with col1:
    st.subheader("Insert PDF")
    uploaded_pdf = st.file_uploader("Upload a PDF file", type="pdf")

# Center Column (Upload DOCX)
with col2:
    st.subheader("Insert Docx")
    uploaded_doc = st.file_uploader("Upload a DOC/DOCX file", type="docx")

# Right Column (Upload PPT)
with col3:
    st.subheader("Insert PPT")
    uploaded_ppt = st.file_uploader("Upload a PPT file", type="pptx")

# Ask a Question
st.subheader("Ask a Question")
question = st.text_input("Enter your question based on uploaded documents:")

if uploaded_pdf or uploaded_doc or uploaded_ppt:
    all_text = ""

    # Extract text from PDF
    if uploaded_pdf:
        pdf_text = ""
        pdf_reader = PdfReader(uploaded_pdf)
        for page in pdf_reader.pages:
            pdf_text += page.extract_text() or ""
        all_text += pdf_text + "\n"

    # Extract text from DOCX
    if uploaded_doc:
        doc_text = ""
        doc_file = Document(uploaded_doc)
        for paragraph in doc_file.paragraphs:
            doc_text += paragraph.text + "\n"
        all_text += doc_text + "\n"

    # Extract text from PPT
    if uploaded_ppt:
        ppt_text = ""
        ppt_file = Presentation(uploaded_ppt)
        for slide in ppt_file.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    ppt_text += shape.text + "\n"
        all_text += ppt_text + "\n"

    # Process the text using LangChain and Cohere
    if question:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, length_function=len, separators=['\n', '\n\n', ' ', '']
        )
        chunks = text_splitter.split_text(text=all_text)
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vectorstore = FAISS.from_texts(chunks, embedding=embeddings)
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        retrieved_docs = retriever.get_relevant_documents(question)
        context = format_docs(retrieved_docs)

        prompt_template = """Answer the question as precisely as possible using the provided context. If the answer is
                            not contained in the context, say "answer not available in context." \n\n
                            Context: \n {context}\n
                            Question: \n {question} \n
                            Answer:"""
        prompt = PromptTemplate.from_template(template=prompt_template)
        full_prompt = prompt.format(context=context, question=question)

        os.environ['COHERE_API_KEY'] = "F0VHvdKjvsfb6jTa9gAtqjL7GlEY7fxdHKwcUwV5"
        cohere_client = cohere.Client(os.getenv('COHERE_API_KEY'))
        response = cohere_client.generate(
            model='command',
            prompt=full_prompt,
            max_tokens=150,
            temperature=0.1
        )

        # Display the answer
        st.write("Answer:")
        st.write(response.generations[0].text.strip())


Writing app.py


In [5]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.13.147.92:8501

⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼your url is: https://evil-lamps-fold.loca.lt
/content/app.py:6: LangChainDeprecationWarning: Importing HuggingFaceEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceEmbeddings
/content/app.py:7: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from la